In [2]:
!pip install numpy librosa ipython pretty_midi soundfile jams tensorflow

In [3]:
import numpy as np
import librosa
import os
from IPython.display import Audio
import pretty_midi
import soundfile as sf
import jams
np.int = int # deprecated np.int

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

midi_dir= './GAPS_audio/midi'

### Creating feature matrices for MIDI data, encoding expressive techniques

In [5]:
def midi_to_jams(midi_path):
    pm = pretty_midi.PrettyMIDI(midi_path)
    guitar_notes = pm.instruments[0].notes
    jam = jams.JAMS()
    
    note_ann = jams.Annotation(namespace='note')  # custom namespace
    for note in guitar_notes:
        note_ann.append(
            time=note.start,
            duration=note.end - note.start,
            value=note.pitch,
            confidence=note.velocity / 127
        )
    jam.annotations.append(note_ann)
    return jam

In [6]:
def print_jam(jam):
    for ann in jam.annotations:
        print(f"Namespace: {ann.namespace}")
        for obs in ann.data:
            print(f"Time: {obs.time:.3f}, Duration: {obs.duration:.3f}, Value: {obs.value}, Confidence: {obs.confidence}")

In [7]:
example_midi = os.path.join(midi_dir, '001_mvswc.mid')
jam = midi_to_jams(example_midi)
print_jam(jam)

Namespace: note
Time: 10.693, Duration: 0.395, Value: 45, Confidence: 0.7874015748031497
Time: 10.820, Duration: 0.268, Value: 52, Confidence: 0.7874015748031497
Time: 10.825, Duration: 0.214, Value: 60, Confidence: 0.7874015748031497
Time: 10.825, Duration: 0.264, Value: 57, Confidence: 0.7874015748031497
Time: 10.830, Duration: 0.259, Value: 64, Confidence: 0.7874015748031497
Time: 10.830, Duration: 0.259, Value: 69, Confidence: 0.7874015748031497
Time: 11.039, Duration: 0.250, Value: 60, Confidence: 0.7874015748031497
Time: 11.368, Duration: 0.118, Value: 64, Confidence: 0.7874015748031497
Time: 11.509, Duration: 0.164, Value: 60, Confidence: 0.7874015748031497
Time: 11.511, Duration: 0.368, Value: 40, Confidence: 0.7874015748031497
Time: 11.668, Duration: 0.211, Value: 64, Confidence: 0.7874015748031497
Time: 11.791, Duration: 0.250, Value: 47, Confidence: 0.7874015748031497
Time: 11.795, Duration: 0.120, Value: 62, Confidence: 0.7874015748031497
Time: 11.925, Duration: 0.116, Valu

In [8]:
# sample techniques to encode
techniques_list = [
    'alternate picking', 'bend', 'hammer on', 'legato', 
    'pull off', 'slide', 'sweep picking', 'tapping', 'vibrato'
]

In [9]:
# sample notes data with expressive techniques
notes = [
    {'pitch': 58, 'time': 0.671, 'duration': 0.139, 'velocity': 0.504},
    {'pitch': 59, 'time': 0.854, 'duration': 0.209, 'velocity': 0.504},
    {'pitch': 61, 'time': 1.107, 'duration': 0.134, 'velocity': 0.504},
]
# Example techniques per note
note_techniques_seq = [
    ['alternate picking'],
    ['bend'],
    ['hammer on', 'vibrato']
]

In [10]:
def note_to_features(note, previous_note, note_techniques):
    """
    note: dict with keys 'pitch', 'time', 'duration', 'velocity'
    previous_note: previous note dict
    note_techniques: list of technique strings for this note
    """
    time_delta = note['time'] - previous_note['time'] if previous_note else 0.0
    technique_flags = [1 if t in note_techniques else 0 for t in techniques_list]
    
    features = [
        note['pitch'],
        note['duration'],
        note['velocity'],
        time_delta
    ] + technique_flags
    
    return features

In [11]:
def midi_to_feature_matrix(midi_path, midi_to_techniques_fn):
    """
    midi_path: path to a MIDI file
    midi_to_techniques_fn: function that returns list of techniques per note
    """
    pm = pretty_midi.PrettyMIDI(midi_path)
    notes = []

    for inst in pm.instruments:
        for n in inst.notes:
            notes.append({
                'pitch': n.pitch,
                'time': n.start,
                'duration': n.end - n.start,
                'velocity': n.velocity / 127.0  # normalize
            })

    notes.sort(key=lambda x: x['time'])
    X = []
    prev_note = None

    for i, note in enumerate(notes):
        note_techniques = midi_to_techniques_fn(i, note)  # user-defined mapping
        feat = note_to_features(note, prev_note, note_techniques)
        X.append(feat)
        prev_note = note

    return np.array(X, dtype=np.float32)

In [12]:
def encode_all_midis(midi_folder, midi_to_techniques_fn):
    feature_matrices = []
    midi_files = [f for f in os.listdir(midi_folder) if f.endswith('.mid')]

    for midi_file in midi_files:
        path = os.path.join(midi_folder, midi_file)
        X = midi_to_feature_matrix(path, midi_to_techniques_fn)
        feature_matrices.append(X)

    return feature_matrices

In [13]:
def dummy_techniques_fn(idx, note):
    # Just cycles through the techniques for demo purposes
    return [techniques_list[idx % len(techniques_list)]]

In [14]:
feature_matrices = encode_all_midis(midi_dir, dummy_techniques_fn)
print("feature matrix for one midi file")
print(feature_matrices[0])

feature matrix for one midi file
[[53.          1.9772727   0.78740156 ...  0.          0.
   0.        ]
 [57.          1.9454546   0.78740156 ...  0.          0.
   0.        ]
 [60.          1.8181819   0.78740156 ...  0.          0.
   0.        ]
 ...
 [48.          4.5227275   0.78740156 ...  0.          0.
   0.        ]
 [53.          4.431818    0.78740156 ...  0.          0.
   0.        ]
 [57.          4.309091    0.78740156 ...  0.          0.
   0.        ]]


In [15]:
import numpy as np

# feature_matrices is a list of arrays of shape (seq_len_i, feature_size)
# First, find the maximum sequence length
max_seq_len = max([fm.shape[0] for fm in feature_matrices])
feature_size = feature_matrices[0].shape[1]

# Pad sequences to max_seq_len
X_sequences = np.zeros((len(feature_matrices), max_seq_len, feature_size))
for i, fm in enumerate(feature_matrices):
    X_sequences[i, :fm.shape[0], :] = fm  # pad remaining with zeros

# Now create y_target for self-supervised learning (string/fret zeros)
y_target = np.concatenate([X_sequences, np.zeros((X_sequences.shape[0], max_seq_len, 2))], axis=-1)

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
import numpy as np
import random

# hyperparameters
seq_len = 16  # max sequence length
feature_size = 13  # input features: pitch, duration, velocity, time_delta, technique flags
output_size = 2   # string & fret
hidden_units = 128
batch_size = 16
num_epochs = 50
max_fret_jump = 5.0

# Guitar tuning (standard)
# E2, A2, D3, G3, B3, E4 in MIDI numbers
tuning = [40, 45, 50, 55, 59, 64]  

def pitch_to_string_fret(pitch, prefer_low_fret=True):
    """
    Map MIDI pitch to a playable string/fret combination.
    Returns (string_index, fret_number)
    Chooses the string with lowest fret possible (or highest if prefer_low_fret=False).
    """
    possibilities = []
    for string_idx, open_pitch in enumerate(tuning):
        fret = pitch - open_pitch
        if 0 <= fret <= 24: 
            possibilities.append((string_idx, fret))
    if not possibilities:
        # pitch out of guitar range: default to string 0, fret 0
        return 0, 0
    if prefer_low_fret:
        return min(possibilities, key=lambda x: x[1])
    else:
        return max(possibilities, key=lambda x: x[1])

# Prepare input sequences
# feature_matrices: list of arrays (seq_len_i, feature_size)
max_seq_len = seq_len
X_sequences = np.zeros((len(feature_matrices), max_seq_len, feature_size))
y_sequences_corrupted = np.zeros((len(feature_matrices), max_seq_len, output_size))
y_sequences_clean = np.zeros((len(feature_matrices), max_seq_len, output_size))

for i, fm in enumerate(feature_matrices):
    length = min(fm.shape[0], max_seq_len)
    X_sequences[i, :length, :] = fm[:length]
    
    for t in range(length):
        pitch = fm[t, 0]  # first feature is pitch
        
        # Clean target (easier - lowest fret)
        s_clean, f_clean = pitch_to_string_fret(int(round(pitch)), prefer_low_fret=True)
        y_sequences_clean[i, t, 0] = s_clean
        y_sequences_clean[i, t, 1] = f_clean
        
        # Corrupted input (harder - sometimes use high fret)
        if random.random() < 0.3:  # 30% corruption rate
            s_hard, f_hard = pitch_to_string_fret(int(round(pitch)), prefer_low_fret=False)
            y_sequences_corrupted[i, t, 0] = s_hard
            y_sequences_corrupted[i, t, 1] = f_hard
        else:
            y_sequences_corrupted[i, t, 0] = s_clean
            y_sequences_corrupted[i, t, 1] = f_clean

# Custom loss function to penalize large hand jumps with fret/string selection
def hand_jump_loss(y_true, y_pred):
    #MSE
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    
    #Minimizing the squared difference in predicted string/fret positions between consecutive notes
    diff = y_pred[:, 1:, :] - y_pred[:, :-1, :]
    jump_penalty = tf.reduce_mean(tf.square(diff))
    
    return mse_loss + jump_penalty / max_fret_jump

# LSTM Model
model = Sequential([
    Masking(mask_value=0., input_shape=(max_seq_len, feature_size)),
    LSTM(hidden_units, return_sequences=True),
    Dense(output_size)  # predicts string & fret only
])
model.compile(optimizer='adam', loss=hand_jump_loss)

model.fit(
    X_sequences,
    y_sequences_clean,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_split=0.1
)

# Predict optimized string/fret
predicted_sf = model.predict(X_sequences)
predicted_strings = np.round(predicted_sf[..., 0]).astype(int)
predicted_frets = np.round(predicted_sf[..., 1]).astype(int)

print("\nComparison:")
print(f"Corrupted avg fret: {y_sequences_corrupted[:, :, 1][y_sequences_corrupted[:, :, 1] > 0].mean():.2f}")
print(f"Clean (target) avg fret: {y_sequences_clean[:, :, 1][y_sequences_clean[:, :, 1] > 0].mean():.2f}")
print(f"Predicted avg fret: {predicted_frets[predicted_frets > 0].mean():.2f}")

2025-12-29 01:47:20.289402: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/data/subhashprasad/miniconda3/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
/data/subhashprasad/miniconda3/lib/python3.13/site-packages/keras/src/layers/core/masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


W0000 00:00:1767001642.591281  172550 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 5.3457 - val_loss: 3.9473
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 3.9899 - val_loss: 3.4609
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 3.4133 - val_loss: 2.9477
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 3.0095 - val_loss: 2.7192
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2.8666 - val_loss: 2.6111
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.7637 - val_loss: 2.5277
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.6988 - val_loss: 2.4695
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.6207 - val_loss: 2.4297
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.5629 - val_loss: 2.3660
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.5054 - val_loss: 2.3667
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.4697 - val_loss: 2.2798
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.3952 - val_l

In [17]:
sample_idx = 1  # Changed from 0

print("Corrupted (harder) strings:", y_sequences_corrupted[sample_idx, :10, 0].astype(int))
print("Corrupted (harder) frets  :", y_sequences_corrupted[sample_idx, :10, 1].astype(int))
print()
print("Clean (target) strings    :", y_sequences_clean[sample_idx, :10, 0].astype(int))
print("Clean (target) frets      :", y_sequences_clean[sample_idx, :10, 1].astype(int))
print()
print("Predicted strings         :", predicted_strings[sample_idx, :10])
print("Predicted frets           :", predicted_frets[sample_idx, :10])

print("\nFeature vectors for this sequence/window:")
print(X_sequences[sample_idx])

Corrupted (harder) strings: [0 0 0 2 4 5 0 1 2 0]
Corrupted (harder) frets  : [ 0  5 10 24  3  2  0  0  0 22]

Clean (target) strings    : [0 1 2 5 4 5 0 1 2 4]
Clean (target) frets      : [ 0  0  0 10  3  2  0  0  0  3]

Predicted strings         : [1 1 2 5 4 5 1 1 2 4]
Predicted frets           : [0 1 1 6 2 2 1 1 2 1]

Feature vectors for this sequence/window:
[[3.80000000e+01 9.93181825e-01 7.87401557e-01 0.00000000e+00
  1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [4.50000000e+01 9.90909100e-01 7.87401557e-01 4.54545440e-03
  0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [5.00000000e+01 9.81818199e-01 7.87401557e-01 9.09090880e-03
  0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [7.40000000e+01 9.6590

In [18]:
def predicted_to_ascii_tab(predicted_sf, notes_per_row=50):
    """
    predicted_sf: array of shape (num_notes, 2)
    notes_per_row: how many notes to show per visual row
    """
    predicted_sf = np.array(predicted_sf)

    
    # Flatten if needed
    if predicted_sf.ndim == 3:
        predicted_sf = predicted_sf.reshape(-1, 2)
    elif predicted_sf.ndim != 2:
        raise ValueError(f"Unexpected predicted_sf shape: {predicted_sf.shape}")
    
    num_strings = 6
    string_names = ['e', 'B', 'G', 'D', 'A', 'E']
    
    ascii_tab = ''
    for start_idx in range(0, len(predicted_sf), notes_per_row):
        row_notes = predicted_sf[start_idx:start_idx+notes_per_row]
        
        # Initialize lines for this row
        row_lines = ['' for _ in range(num_strings)]
        
        for note in row_notes:
            string, fret = note
            string_idx = int(round(string)) - 1  # 0-indexed
            fret_str = str(int(round(fret)))
            
            for i in range(num_strings):
                if i == string_idx:
                    row_lines[num_strings - i - 1] += fret_str + '-'
                else:
                    row_lines[num_strings - i - 1] += '--'
        
        # Append row to ascii_tab
        for name, line in zip(string_names, row_lines):
            ascii_tab += f"{name}|{line}\n"
        ascii_tab += '\n'  # extra blank line between tab rows
    
    return ascii_tab

In [19]:
ascii_tab = predicted_to_ascii_tab(predicted_sf)
print(ascii_tab)

e|----------------------------------------------------------------------------------------------------
B|--------3---------3---------3---------6---2-----------3---------------------------------------------
G|----1-2-------1-1-------1-1-------------2---------1-2---------1-------------------------------------
D|2-1-------2-1-------2-1-------2-----------------------------------------------------------------2---
A|------------------------------------1-----------2---------1-----1-1-1-----1-1-1-2-2-----1---1-------
E|--------------------------------0-1---------1-1---------1---1---------1-2-----------1-2---1---1---1-

e|----------------------------------------------------------------------------------------------------
B|----------------------------4-7-10-10-10---6-7-8-7-7---8-10-10-10---------------------------------------2-
G|------------------------------------------------------------------------------------1---1-1---------
D|----1-----------------1---1---------------------------------2-1-

In [ ]:
# Evaluation metrics

def evaluate_tabs(tab_sf, name=""):
    strings = tab_sf[:, :, 0]
    frets = tab_sf[:, :, 1]
    active_frets = frets[frets > 0]
    
    print(f"\n{name}")
    
    # Fret position
    avg_fret = active_frets.mean() if len(active_frets) > 0 else 0
    max_fret = active_frets.max() if len(active_frets) > 0 else 0
    open_strings = np.sum(frets == 0)
    total_notes = np.sum(frets >= 0)
    open_pct = (open_strings / total_notes * 100) if total_notes > 0 else 0
    
    print(f"Fret Position: avg={avg_fret:.2f}, max={max_fret:.0f}, open={open_pct:.1f}%")
    
    # Jump length
    jumps = []
    for seq_idx in range(tab_sf.shape[0]):
        prev_fret = None
        for t in range(tab_sf.shape[1]):
            curr_fret = frets[seq_idx, t]
            if curr_fret > 0:
                if prev_fret is not None and prev_fret > 0:
                    jumps.append(abs(curr_fret - prev_fret))
                prev_fret = curr_fret
    
    avg_jump = np.mean(jumps) if jumps else 0
    max_jump = np.max(jumps) if jumps else 0
    large_jumps = np.sum(np.array(jumps) > 5) if jumps else 0
    
    print(f"Jump Length: avg={avg_jump:.2f}, max={max_jump:.0f}, large(>5)={large_jumps}")
    
    # Hand stretch
    stretches = []
    for seq_idx in range(tab_sf.shape[0]):
        for t in range(tab_sf.shape[1] - 3):
            window_frets = frets[seq_idx, t:t+4]
            active = window_frets[window_frets > 0]
            if len(active) > 1:
                stretches.append(active.max() - active.min())
    
    avg_stretch = np.mean(stretches) if stretches else 0
    max_stretch = np.max(stretches) if stretches else 0
    hard_stretches = np.sum(np.array(stretches) > 4) if stretches else 0
    
    print(f"Hand Stretch: avg={avg_stretch:.2f}, max={max_stretch:.0f}, difficult(>4)={hard_stretches}")
    
    # Bar chords
    bar_chords = 0
    for seq_idx in range(tab_sf.shape[0]):
        for t in range(tab_sf.shape[1] - 1):
            curr_fret = frets[seq_idx, t]
            next_fret = frets[seq_idx, t+1]
            curr_string = strings[seq_idx, t]
            next_string = strings[seq_idx, t+1]
            
            if curr_fret > 3 and curr_fret == next_fret and curr_string != next_string:
                bar_chords += 1
    
    print(f"Bar Chords: {bar_chords}")
    
    return {'avg_fret': avg_fret, 'avg_jump': avg_jump, 'avg_stretch': avg_stretch, 'bar_chords': bar_chords}

corrupted_metrics = evaluate_tabs(y_sequences_corrupted, "CORRUPTED")
predicted_metrics = evaluate_tabs(predicted_sf, "PREDICTED")
clean_metrics = evaluate_tabs(y_sequences_clean, "CLEAN")

print("\nIMPROVEMENT:")

def calc_improvement(corrupted, predicted, clean):
    if abs(corrupted - clean) < 0.01:
        return 0
    return (corrupted - predicted) / (corrupted - clean) * 100

for metric in ['avg_fret', 'avg_jump', 'avg_stretch']:
    imp = calc_improvement(corrupted_metrics[metric], predicted_metrics[metric], clean_metrics[metric])
    status = "GOOD" if imp > 50 else "BAD"
    print(f"{metric}: {imp:.0f}% {status}")